# Introduction

This notebook builds and evaluates a model to classify customer behavior as fraudulent or not.

# Setup

Navigate to project's root directory:

In [1]:
%cd ..

/tf


# Libraries

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:
tf.__version__

'2.0.0-beta1'

In [4]:
tf.random.set_seed(123)

# Data

In [5]:
train_df = pd.read_parquet("./data/train_feat.parquet")
valid_df = pd.read_parquet("./data/valid_feat.parquet")
fraud_df = pd.read_parquet("./data/fraud_feat.parquet")

test_df = pd.read_parquet("./data/test_feat.parquet")

In [6]:
feature_cols = [f"Behavior{i}" for i in range(1, 12)]
feature_cols

['Behavior1',
 'Behavior2',
 'Behavior3',
 'Behavior4',
 'Behavior5',
 'Behavior6',
 'Behavior7',
 'Behavior8',
 'Behavior9',
 'Behavior10',
 'Behavior11']

In [7]:
num_features = len(feature_cols)
num_features

11

In [8]:
num_fraud, _ = fraud_df.shape
num_fraud

7

In [9]:
eval_df = valid_df.sample(n=num_fraud, random_state=123)
eval_df

,Behavior1,Behavior2,Behavior3,Behavior4,Behavior5,Behavior6,Behavior7,Behavior8,Behavior9,Behavior10,Behavior11
50,-1.116949,-0.284010,-1.070120,-0.827804,0.169460,1.381103,0.715423,0.479916,1.166299,-1.049028,-0.863424
127,0.685185,-1.455636,-0.763855,0.910445,2.492966,-1.559258,-0.738419,-0.556776,1.076575,-0.469531,-0.599081
37,0.882029,0.929629,-0.199266,1.172973,0.270157,0.804269,0.087448,0.177107,1.164299,-1.382487,-0.409866
149,1.965932,1.088959,0.954181,-1.865497,-0.236305,1.540129,-1.697855,-0.002534,1.047617,0.836702,0.179371
19,0.337095,1.921309,0.445954,0.775971,2.157673,-1.365846,1.344258,-0.228552,1.661230,0.573365,0.135853
104,0.642768,0.073340,0.026926,-0.690515,0.937142,0.661341,-0.607261,-0.550967,-0.701392,-0.668839,0.630568
179,-0.414703,1.103888,1.693856,-0.757967,-0.175606,-0.033972,1.084393,0.013140,1.019464,0.246194,-1.160555


# Model

Function to evaluate a model in terms of its reconstruction error:

In [10]:
def evaluate_model(model):
    eval_features = eval_df[feature_cols].values
    fraud_features = fraud_df[feature_cols].values
    
    loss_eval, _ = model.evaluate(x= eval_features,
                                  y= eval_features)

    loss_fraud, _ = model.evaluate(x=fraud_features,
                                   y=fraud_features)

    print(f"Average loss on the eval set:\t{loss_eval:>7.4f}\nAverage loss on fraud set:\t {loss_fraud:.4f}")

Function to compute the reconstruction loss:

In [11]:
def compute_reconstruction_loss(model, df):
    features = df[feature_cols].values
    predictions = model.predict(features)
    losses = tf.keras.losses.mean_squared_error(y_true=features,
                                                y_pred=predictions)
    
    return pd.DataFrame({"reconstruction_loss": losses})
    

Baseline:

In [12]:
%%time
baseline = tf.keras.Sequential([
    tf.keras.layers.Input([num_features]),
    tf.keras.layers.Dense(6, activation=tf.keras.layers.LeakyReLU()),
    tf.keras.layers.Dense(num_features),
])

baseline.compile(optimizer='adam',
                 loss='mean_squared_error',
                 metrics=['mean_squared_error'])

baseline.fit(x=train_df[feature_cols], 
             y=train_df[feature_cols],  
             epochs=500,
             validation_data=(valid_df[feature_cols], valid_df[feature_cols])
            )

Train on 800 samples, validate on 200 samples
Epoch 1/500
800/800 [==============================] - 0s 293us/sample - loss: 1.1860 - mean_squared_error: 1.1860 - val_loss: 1.1496 - val_mean_squared_error: 1.1496
Epoch 2/500
800/800 [==============================] - 0s 67us/sample - loss: 1.0675 - mean_squared_error: 1.0675 - val_loss: 1.0467 - val_mean_squared_error: 1.0467
Epoch 3/500
800/800 [==============================] - 0s 78us/sample - loss: 0.9771 - mean_squared_error: 0.9771 - val_loss: 0.9687 - val_mean_squared_error: 0.9687
Epoch 4/500
800/800 [==============================] - 0s 132us/sample - loss: 0.9079 - mean_squared_error: 0.9079 - val_loss: 0.9076 - val_mean_squared_error: 0.9076
Epoch 5/500
800/800 [==============================] - 0s 107us/sample - loss: 0.8518 - mean_squared_error: 0.8518 - val_loss: 0.8587 - val_mean_squared_error: 0.8587
Epoch 6/500
800/800 [==============================] - 0s 62us/sample - loss: 0.8064 - mean_squared_error: 0.8064 - val_l

Epoch 50/500
800/800 [==============================] - 0s 49us/sample - loss: 0.4552 - mean_squared_error: 0.4552 - val_loss: 0.5043 - val_mean_squared_error: 0.5043
Epoch 51/500
800/800 [==============================] - 0s 61us/sample - loss: 0.4545 - mean_squared_error: 0.4545 - val_loss: 0.5034 - val_mean_squared_error: 0.5034
Epoch 52/500
800/800 [==============================] - 0s 50us/sample - loss: 0.4537 - mean_squared_error: 0.4537 - val_loss: 0.5029 - val_mean_squared_error: 0.5029
Epoch 53/500
800/800 [==============================] - 0s 52us/sample - loss: 0.4529 - mean_squared_error: 0.4529 - val_loss: 0.5022 - val_mean_squared_error: 0.5022
Epoch 54/500
800/800 [==============================] - 0s 72us/sample - loss: 0.4523 - mean_squared_error: 0.4523 - val_loss: 0.5016 - val_mean_squared_error: 0.5016
Epoch 55/500
800/800 [==============================] - 0s 53us/sample - loss: 0.4516 - mean_squared_error: 0.4516 - val_loss: 0.5010 - val_mean_squared_error: 0.501

800/800 [==============================] - 0s 52us/sample - loss: 0.4241 - mean_squared_error: 0.4241 - val_loss: 0.4723 - val_mean_squared_error: 0.4723
Epoch 100/500
800/800 [==============================] - 0s 60us/sample - loss: 0.4234 - mean_squared_error: 0.4234 - val_loss: 0.4720 - val_mean_squared_error: 0.4720
Epoch 101/500
800/800 [==============================] - 0s 70us/sample - loss: 0.4227 - mean_squared_error: 0.4227 - val_loss: 0.4708 - val_mean_squared_error: 0.4708
Epoch 102/500
800/800 [==============================] - 0s 68us/sample - loss: 0.4221 - mean_squared_error: 0.4221 - val_loss: 0.4707 - val_mean_squared_error: 0.4707
Epoch 103/500
800/800 [==============================] - 0s 45us/sample - loss: 0.4215 - mean_squared_error: 0.4215 - val_loss: 0.4698 - val_mean_squared_error: 0.4698
Epoch 104/500
800/800 [==============================] - 0s 53us/sample - loss: 0.4210 - mean_squared_error: 0.4210 - val_loss: 0.4691 - val_mean_squared_error: 0.4691
Epoch 

Epoch 148/500
800/800 [==============================] - 0s 59us/sample - loss: 0.3978 - mean_squared_error: 0.3978 - val_loss: 0.4422 - val_mean_squared_error: 0.4422
Epoch 149/500
800/800 [==============================] - 0s 51us/sample - loss: 0.3973 - mean_squared_error: 0.3973 - val_loss: 0.4413 - val_mean_squared_error: 0.4413
Epoch 150/500
800/800 [==============================] - 0s 57us/sample - loss: 0.3971 - mean_squared_error: 0.3971 - val_loss: 0.4407 - val_mean_squared_error: 0.4407
Epoch 151/500
800/800 [==============================] - 0s 71us/sample - loss: 0.3968 - mean_squared_error: 0.3968 - val_loss: 0.4407 - val_mean_squared_error: 0.4407
Epoch 152/500
800/800 [==============================] - 0s 69us/sample - loss: 0.3965 - mean_squared_error: 0.3965 - val_loss: 0.4404 - val_mean_squared_error: 0.4404
Epoch 153/500
800/800 [==============================] - 0s 49us/sample - loss: 0.3962 - mean_squared_error: 0.3962 - val_loss: 0.4393 - val_mean_squared_error:

Epoch 197/500
800/800 [==============================] - 0s 92us/sample - loss: 0.3868 - mean_squared_error: 0.3868 - val_loss: 0.4275 - val_mean_squared_error: 0.4275
Epoch 198/500
800/800 [==============================] - 0s 154us/sample - loss: 0.3869 - mean_squared_error: 0.3869 - val_loss: 0.4268 - val_mean_squared_error: 0.4268
Epoch 199/500
800/800 [==============================] - 0s 98us/sample - loss: 0.3865 - mean_squared_error: 0.3865 - val_loss: 0.4273 - val_mean_squared_error: 0.4273
Epoch 200/500
800/800 [==============================] - 0s 56us/sample - loss: 0.3868 - mean_squared_error: 0.3868 - val_loss: 0.4270 - val_mean_squared_error: 0.4270
Epoch 201/500
800/800 [==============================] - 0s 57us/sample - loss: 0.3864 - mean_squared_error: 0.3864 - val_loss: 0.4272 - val_mean_squared_error: 0.4272
Epoch 202/500
800/800 [==============================] - 0s 69us/sample - loss: 0.3862 - mean_squared_error: 0.3862 - val_loss: 0.4262 - val_mean_squared_error

...

Epoch 294/500
800/800 [==============================] - 0s 178us/sample - loss: 0.3818 - mean_squared_error: 0.3818 - val_loss: 0.4200 - val_mean_squared_error: 0.4200
Epoch 295/500
800/800 [==============================] - 0s 189us/sample - loss: 0.3817 - mean_squared_error: 0.3817 - val_loss: 0.4196 - val_mean_squared_error: 0.4196
Epoch 296/500
800/800 [==============================] - 0s 121us/sample - loss: 0.3819 - mean_squared_error: 0.3819 - val_loss: 0.4199 - val_mean_squared_error: 0.4199
Epoch 297/500
800/800 [==============================] - 0s 128us/sample - loss: 0.3818 - mean_squared_error: 0.3818 - val_loss: 0.4202 - val_mean_squared_error: 0.4202
Epoch 298/500
800/800 [==============================] - 0s 104us/sample - loss: 0.3820 - mean_squared_error: 0.3820 - val_loss: 0.4206 - val_mean_squared_error: 0.4206
Epoch 299/500
800/800 [==============================] - 0s 80us/sample - loss: 0.3816 - mean_squared_error: 0.3816 - val_loss: 0.4198 - val_mean_squared_e

800/800 [==============================] - 0s 110us/sample - loss: 0.3807 - mean_squared_error: 0.3807 - val_loss: 0.4184 - val_mean_squared_error: 0.4184
Epoch 343/500
800/800 [==============================] - 0s 114us/sample - loss: 0.3811 - mean_squared_error: 0.3811 - val_loss: 0.4187 - val_mean_squared_error: 0.4187
Epoch 344/500
800/800 [==============================] - 0s 108us/sample - loss: 0.3807 - mean_squared_error: 0.3807 - val_loss: 0.4182 - val_mean_squared_error: 0.4182
Epoch 345/500
800/800 [==============================] - 0s 120us/sample - loss: 0.3811 - mean_squared_error: 0.3811 - val_loss: 0.4191 - val_mean_squared_error: 0.4191
Epoch 346/500
800/800 [==============================] - 0s 208us/sample - loss: 0.3810 - mean_squared_error: 0.3810 - val_loss: 0.4185 - val_mean_squared_error: 0.4185
Epoch 347/500
800/800 [==============================] - 0s 137us/sample - loss: 0.3807 - mean_squared_error: 0.3807 - val_loss: 0.4188 - val_mean_squared_error: 0.4188


Epoch 391/500
800/800 [==============================] - 0s 109us/sample - loss: 0.3804 - mean_squared_error: 0.3804 - val_loss: 0.4182 - val_mean_squared_error: 0.4182
Epoch 392/500
800/800 [==============================] - 0s 97us/sample - loss: 0.3803 - mean_squared_error: 0.3803 - val_loss: 0.4172 - val_mean_squared_error: 0.4172
Epoch 393/500
800/800 [==============================] - 0s 84us/sample - loss: 0.3808 - mean_squared_error: 0.3808 - val_loss: 0.4182 - val_mean_squared_error: 0.4182
Epoch 394/500
800/800 [==============================] - 0s 119us/sample - loss: 0.3803 - mean_squared_error: 0.3803 - val_loss: 0.4174 - val_mean_squared_error: 0.4174
Epoch 395/500
800/800 [==============================] - 0s 124us/sample - loss: 0.3809 - mean_squared_error: 0.3809 - val_loss: 0.4172 - val_mean_squared_error: 0.4172
Epoch 396/500
800/800 [==============================] - 0s 118us/sample - loss: 0.3807 - mean_squared_error: 0.3807 - val_loss: 0.4179 - val_mean_squared_er

Epoch 440/500
800/800 [==============================] - 0s 52us/sample - loss: 0.3800 - mean_squared_error: 0.3800 - val_loss: 0.4166 - val_mean_squared_error: 0.4166
Epoch 441/500
800/800 [==============================] - 0s 49us/sample - loss: 0.3805 - mean_squared_error: 0.3805 - val_loss: 0.4174 - val_mean_squared_error: 0.4174
Epoch 442/500
800/800 [==============================] - 0s 66us/sample - loss: 0.3804 - mean_squared_error: 0.3804 - val_loss: 0.4169 - val_mean_squared_error: 0.4169
Epoch 443/500
800/800 [==============================] - 0s 57us/sample - loss: 0.3803 - mean_squared_error: 0.3803 - val_loss: 0.4172 - val_mean_squared_error: 0.4172
Epoch 444/500
800/800 [==============================] - 0s 52us/sample - loss: 0.3802 - mean_squared_error: 0.3802 - val_loss: 0.4170 - val_mean_squared_error: 0.4170
Epoch 445/500
800/800 [==============================] - 0s 54us/sample - loss: 0.3801 - mean_squared_error: 0.3801 - val_loss: 0.4172 - val_mean_squared_error:

Epoch 489/500
800/800 [==============================] - 0s 52us/sample - loss: 0.3801 - mean_squared_error: 0.3801 - val_loss: 0.4167 - val_mean_squared_error: 0.4167
Epoch 490/500
800/800 [==============================] - 0s 95us/sample - loss: 0.3797 - mean_squared_error: 0.3797 - val_loss: 0.4170 - val_mean_squared_error: 0.4170
Epoch 491/500
800/800 [==============================] - 0s 96us/sample - loss: 0.3803 - mean_squared_error: 0.3803 - val_loss: 0.4161 - val_mean_squared_error: 0.4161
Epoch 492/500
800/800 [==============================] - 0s 45us/sample - loss: 0.3796 - mean_squared_error: 0.3796 - val_loss: 0.4166 - val_mean_squared_error: 0.4166
Epoch 493/500
800/800 [==============================] - 0s 44us/sample - loss: 0.3794 - mean_squared_error: 0.3794 - val_loss: 0.4167 - val_mean_squared_error: 0.4167
Epoch 494/500
800/800 [==============================] - 0s 46us/sample - loss: 0.3796 - mean_squared_error: 0.3796 - val_loss: 0.4169 - val_mean_squared_error:

In [13]:
evaluate_model(baseline)

7/7 [==============================] - 0s 127us/sample - loss: 1.4843 - mean_squared_error: 1.4843
Average loss on the eval set:	 0.4105
Average loss on fraud set:	 1.4843


In [14]:
baseline_eval = compute_reconstruction_loss(baseline, eval_df)
baseline_fraud = compute_reconstruction_loss(baseline, fraud_df)

Autoencoder:

In [15]:
encoder_outputs = [128, 6]
decoder_outputs = encoder_outputs[1::-1] if len(encoder_outputs) > 1 else []

def build_dense_layer(layer_name, output_dimension):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(output_dimension, activation=None),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('elu')
    ],
    name=layer_name)

encoder_layers = [build_dense_layer(f"encoder_layer_{i}", output_dim)
                  for i, output_dim in enumerate(encoder_outputs)]

encoder_layers.insert(0, tf.keras.layers.Input([num_features]))

encoder_model = tf.keras.Sequential(encoder_layers, name="encoder")



decoder_layers = [build_dense_layer(f"decoder_layer_{i}", output_dim)
                 for i, output_dim in enumerate(decoder_outputs)]

encoder_output_shape = encoder_model.layers[-1].output.shape.as_list()[1:]

decoder_layers.insert(0, tf.keras.layers.Input(encoder_output_shape))

decoder_model = tf.keras.Sequential(decoder_layers, name="decoder")

output_layer = tf.keras.Sequential([
        tf.keras.layers.Dense(num_features)
    ],
    name="output_layer")

model = tf.keras.Sequential([
        encoder_model,
        decoder_model,
        output_layer
    ],
    name="autoencoder")


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

In [16]:
%%time
num_epochs = 700
model.fit(x=train_df[feature_cols].values, 
          y=train_df[feature_cols].values,  
          epochs=num_epochs,
          validation_data=(valid_df[feature_cols].values, 
                           valid_df[feature_cols].values),
          batch_size=800
         )

Train on 800 samples, validate on 200 samples
Epoch 1/700
800/800 [==============================] - 1s 1ms/sample - loss: 1.9007 - mean_squared_error: 1.9007 - val_loss: 1.0097 - val_mean_squared_error: 1.0097
Epoch 2/700
800/800 [==============================] - 0s 15us/sample - loss: 1.6899 - mean_squared_error: 1.6899 - val_loss: 0.9992 - val_mean_squared_error: 0.9992
Epoch 3/700
800/800 [==============================] - 0s 14us/sample - loss: 1.5106 - mean_squared_error: 1.5106 - val_loss: 0.9905 - val_mean_squared_error: 0.9905
Epoch 4/700
800/800 [==============================] - 0s 16us/sample - loss: 1.3608 - mean_squared_error: 1.3608 - val_loss: 0.9835 - val_mean_squared_error: 0.9835
Epoch 5/700
800/800 [==============================] - 0s 15us/sample - loss: 1.2380 - mean_squared_error: 1.2380 - val_loss: 0.9781 - val_mean_squared_error: 0.9781
Epoch 6/700
800/800 [==============================] - 0s 16us/sample - loss: 1.1390 - mean_squared_error: 1.1390 - val_loss:

Epoch 50/700
800/800 [==============================] - 0s 18us/sample - loss: 0.5874 - mean_squared_error: 0.5874 - val_loss: 0.8970 - val_mean_squared_error: 0.8970
Epoch 51/700
800/800 [==============================] - 0s 15us/sample - loss: 0.5847 - mean_squared_error: 0.5847 - val_loss: 0.8957 - val_mean_squared_error: 0.8957
Epoch 52/700
800/800 [==============================] - 0s 17us/sample - loss: 0.5823 - mean_squared_error: 0.5823 - val_loss: 0.8943 - val_mean_squared_error: 0.8943
Epoch 53/700
800/800 [==============================] - 0s 20us/sample - loss: 0.5800 - mean_squared_error: 0.5800 - val_loss: 0.8928 - val_mean_squared_error: 0.8928
Epoch 54/700
800/800 [==============================] - 0s 18us/sample - loss: 0.5778 - mean_squared_error: 0.5778 - val_loss: 0.8914 - val_mean_squared_error: 0.8914
Epoch 55/700
800/800 [==============================] - 0s 19us/sample - loss: 0.5758 - mean_squared_error: 0.5758 - val_loss: 0.8899 - val_mean_squared_error: 0.889

800/800 [==============================] - 0s 20us/sample - loss: 0.5324 - mean_squared_error: 0.5324 - val_loss: 0.8283 - val_mean_squared_error: 0.8283
Epoch 100/700
800/800 [==============================] - 0s 30us/sample - loss: 0.5318 - mean_squared_error: 0.5318 - val_loss: 0.8270 - val_mean_squared_error: 0.8270
Epoch 101/700
800/800 [==============================] - 0s 17us/sample - loss: 0.5312 - mean_squared_error: 0.5312 - val_loss: 0.8257 - val_mean_squared_error: 0.8257
Epoch 102/700
800/800 [==============================] - 0s 17us/sample - loss: 0.5307 - mean_squared_error: 0.5307 - val_loss: 0.8244 - val_mean_squared_error: 0.8244
Epoch 103/700
800/800 [==============================] - 0s 20us/sample - loss: 0.5301 - mean_squared_error: 0.5301 - val_loss: 0.8231 - val_mean_squared_error: 0.8231
Epoch 104/700
800/800 [==============================] - 0s 19us/sample - loss: 0.5295 - mean_squared_error: 0.5295 - val_loss: 0.8218 - val_mean_squared_error: 0.8218
Epoch 

Epoch 148/700
800/800 [==============================] - 0s 36us/sample - loss: 0.5052 - mean_squared_error: 0.5052 - val_loss: 0.7654 - val_mean_squared_error: 0.7654
Epoch 149/700
800/800 [==============================] - 0s 33us/sample - loss: 0.5044 - mean_squared_error: 0.5044 - val_loss: 0.7640 - val_mean_squared_error: 0.7640
Epoch 150/700
800/800 [==============================] - 0s 34us/sample - loss: 0.5036 - mean_squared_error: 0.5036 - val_loss: 0.7627 - val_mean_squared_error: 0.7627
Epoch 151/700
800/800 [==============================] - 0s 43us/sample - loss: 0.5027 - mean_squared_error: 0.5027 - val_loss: 0.7613 - val_mean_squared_error: 0.7613
Epoch 152/700
800/800 [==============================] - 0s 35us/sample - loss: 0.5018 - mean_squared_error: 0.5018 - val_loss: 0.7599 - val_mean_squared_error: 0.7599
Epoch 153/700
800/800 [==============================] - 0s 34us/sample - loss: 0.5008 - mean_squared_error: 0.5008 - val_loss: 0.7585 - val_mean_squared_error:

Epoch 197/700
800/800 [==============================] - 0s 22us/sample - loss: 0.4378 - mean_squared_error: 0.4378 - val_loss: 0.6687 - val_mean_squared_error: 0.6687
Epoch 198/700
800/800 [==============================] - 0s 31us/sample - loss: 0.4370 - mean_squared_error: 0.4370 - val_loss: 0.6669 - val_mean_squared_error: 0.6669
Epoch 199/700
800/800 [==============================] - 0s 38us/sample - loss: 0.4362 - mean_squared_error: 0.4362 - val_loss: 0.6652 - val_mean_squared_error: 0.6652
Epoch 200/700
800/800 [==============================] - 0s 29us/sample - loss: 0.4355 - mean_squared_error: 0.4355 - val_loss: 0.6635 - val_mean_squared_error: 0.6635
Epoch 201/700
800/800 [==============================] - 0s 17us/sample - loss: 0.4347 - mean_squared_error: 0.4347 - val_loss: 0.6618 - val_mean_squared_error: 0.6618
Epoch 202/700
800/800 [==============================] - 0s 27us/sample - loss: 0.4340 - mean_squared_error: 0.4340 - val_loss: 0.6602 - val_mean_squared_error:

...

Epoch 491/700
800/800 [==============================] - 0s 19us/sample - loss: 0.3294 - mean_squared_error: 0.3294 - val_loss: 0.4561 - val_mean_squared_error: 0.4561
Epoch 492/700
800/800 [==============================] - 0s 20us/sample - loss: 0.3292 - mean_squared_error: 0.3292 - val_loss: 0.4559 - val_mean_squared_error: 0.4559
Epoch 493/700
800/800 [==============================] - 0s 19us/sample - loss: 0.3290 - mean_squared_error: 0.3290 - val_loss: 0.4557 - val_mean_squared_error: 0.4557
Epoch 494/700
800/800 [==============================] - 0s 16us/sample - loss: 0.3288 - mean_squared_error: 0.3288 - val_loss: 0.4556 - val_mean_squared_error: 0.4556
Epoch 495/700
800/800 [==============================] - 0s 14us/sample - loss: 0.3285 - mean_squared_error: 0.3285 - val_loss: 0.4554 - val_mean_squared_error: 0.4554
Epoch 496/700
800/800 [==============================] - 0s 22us/sample - loss: 0.3283 - mean_squared_error: 0.3283 - val_loss: 0.4552 - val_mean_squared_error:

Epoch 540/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3195 - mean_squared_error: 0.3195 - val_loss: 0.4488 - val_mean_squared_error: 0.4488
Epoch 541/700
800/800 [==============================] - 0s 29us/sample - loss: 0.3193 - mean_squared_error: 0.3193 - val_loss: 0.4487 - val_mean_squared_error: 0.4487
Epoch 542/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3191 - mean_squared_error: 0.3191 - val_loss: 0.4486 - val_mean_squared_error: 0.4486
Epoch 543/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3190 - mean_squared_error: 0.3190 - val_loss: 0.4485 - val_mean_squared_error: 0.4485
Epoch 544/700
800/800 [==============================] - 0s 19us/sample - loss: 0.3188 - mean_squared_error: 0.3188 - val_loss: 0.4483 - val_mean_squared_error: 0.4483
Epoch 545/700
800/800 [==============================] - 0s 19us/sample - loss: 0.3186 - mean_squared_error: 0.3186 - val_loss: 0.4482 - val_mean_squared_error:

Epoch 589/700
800/800 [==============================] - 0s 18us/sample - loss: 0.3106 - mean_squared_error: 0.3106 - val_loss: 0.4435 - val_mean_squared_error: 0.4435
Epoch 590/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3104 - mean_squared_error: 0.3104 - val_loss: 0.4434 - val_mean_squared_error: 0.4434
Epoch 591/700
800/800 [==============================] - 0s 20us/sample - loss: 0.3102 - mean_squared_error: 0.3102 - val_loss: 0.4434 - val_mean_squared_error: 0.4434
Epoch 592/700
800/800 [==============================] - 0s 18us/sample - loss: 0.3101 - mean_squared_error: 0.3101 - val_loss: 0.4433 - val_mean_squared_error: 0.4433
Epoch 593/700
800/800 [==============================] - 0s 20us/sample - loss: 0.3099 - mean_squared_error: 0.3099 - val_loss: 0.4432 - val_mean_squared_error: 0.4432
Epoch 594/700
800/800 [==============================] - 0s 19us/sample - loss: 0.3097 - mean_squared_error: 0.3097 - val_loss: 0.4431 - val_mean_squared_error:

Epoch 638/700
800/800 [==============================] - 0s 17us/sample - loss: 0.3022 - mean_squared_error: 0.3022 - val_loss: 0.4395 - val_mean_squared_error: 0.4395
Epoch 639/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3020 - mean_squared_error: 0.3020 - val_loss: 0.4394 - val_mean_squared_error: 0.4394
Epoch 640/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3018 - mean_squared_error: 0.3018 - val_loss: 0.4393 - val_mean_squared_error: 0.4393
Epoch 641/700
800/800 [==============================] - 0s 16us/sample - loss: 0.3017 - mean_squared_error: 0.3017 - val_loss: 0.4392 - val_mean_squared_error: 0.4392
Epoch 642/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3015 - mean_squared_error: 0.3015 - val_loss: 0.4392 - val_mean_squared_error: 0.4392
Epoch 643/700
800/800 [==============================] - 0s 15us/sample - loss: 0.3013 - mean_squared_error: 0.3013 - val_loss: 0.4391 - val_mean_squared_error:

Epoch 687/700
800/800 [==============================] - 0s 26us/sample - loss: 0.2940 - mean_squared_error: 0.2940 - val_loss: 0.4359 - val_mean_squared_error: 0.4359
Epoch 688/700
800/800 [==============================] - 0s 24us/sample - loss: 0.2938 - mean_squared_error: 0.2938 - val_loss: 0.4358 - val_mean_squared_error: 0.4358
Epoch 689/700
800/800 [==============================] - 0s 26us/sample - loss: 0.2937 - mean_squared_error: 0.2937 - val_loss: 0.4357 - val_mean_squared_error: 0.4357
Epoch 690/700
800/800 [==============================] - 0s 32us/sample - loss: 0.2935 - mean_squared_error: 0.2935 - val_loss: 0.4356 - val_mean_squared_error: 0.4356
Epoch 691/700
800/800 [==============================] - 0s 26us/sample - loss: 0.2933 - mean_squared_error: 0.2933 - val_loss: 0.4356 - val_mean_squared_error: 0.4356
Epoch 692/700
800/800 [==============================] - 0s 22us/sample - loss: 0.2932 - mean_squared_error: 0.2932 - val_loss: 0.4355 - val_mean_squared_error:

In [17]:
evaluate_model(model)

7/7 [==============================] - 0s 229us/sample - loss: 1.7795 - mean_squared_error: 1.7795
Average loss on the eval set:	 0.3731
Average loss on fraud set:	 1.7795


In [18]:
model_eval = compute_reconstruction_loss(model, eval_df)
model_fraud = compute_reconstruction_loss(model, fraud_df)

Compare the reconstruction losses from baseline and autoencoder model:

In [19]:
loss_eval = pd.concat([
    baseline_eval,
    model_eval
],
axis=1)

loss_eval.columns = ["baseline", "autoencoder"]

loss_eval

,baseline,autoencoder
0,0.233120,0.070637
1,0.934373,0.969130
2,0.317213,0.325807
3,0.300937,0.422348
4,0.822465,0.504984
5,0.109636,0.090340
6,0.155583,0.228394


In [20]:
loss_eval.describe()

,baseline,autoencoder
count,7.000000,7.000000
mean,0.410475,0.373091
std,0.329604,0.308135
min,0.109636,0.070637
25%,0.194351,0.159367
50%,0.300937,0.325807
75%,0.569839,0.463666
max,0.934373,0.969130


In [21]:
loss_fraud = pd.concat([
    baseline_fraud,
    model_fraud
],
axis=1)

loss_fraud.columns = ["baseline", "autoencoder"]

loss_fraud

,baseline,autoencoder
0,1.129046,0.784801
1,0.364508,0.811793
2,2.320477,2.932226
3,1.801669,2.125167
4,0.330132,0.578325
5,1.395267,1.461898
6,3.049002,3.761956


In [22]:
loss_fraud.describe()

,baseline,autoencoder
count,7.000000,7.000000
mean,1.484300,1.779452
std,0.997361,1.214357
min,0.330132,0.578325
25%,0.746777,0.798297
50%,1.395267,1.461898
75%,2.061073,2.528696
max,3.049002,3.761956


# Predicting on the Test Set

Create a function to normalize the reconstruction loss:

In [23]:
def build_loss_scaler(model, df):
    recon_loss = compute_reconstruction_loss(model, df)
    scaler = MinMaxScaler()
    scaler.fit(recon_loss)
    
    return scaler

Function to compute the fraud score of a customer:

In [24]:
def compute_fraud_score(model, df, scaler):
    predictions = model.predict(df[feature_cols].values)
    recon_loss = compute_reconstruction_loss(model, df)
    
    fraud_scores = pd.DataFrame.from_records(scaler.transform(recon_loss), 
                                             columns=["fraud_score"])
    
    return fraud_scores

Compute fraud score for the train, valid and fraud set:

In [25]:
scaler = build_loss_scaler(model, train_df)

In [26]:
compute_fraud_score(model, train_df, scaler).describe()

,fraud_score
count,800.000000
mean,0.241012
std,0.155532
min,0.000000
25%,0.129440
50%,0.203142
75%,0.314493
max,1.000000


In [27]:
compute_fraud_score(model, train_df, scaler).query('fraud_score > 0.8')

,fraud_score
119,0.877574
124,0.820826
141,0.836677
305,1.000000
697,0.813874


In [28]:
compute_fraud_score(model, valid_df, scaler).describe()

,fraud_score
count,200.000000
mean,0.328893
std,0.220852
min,0.029441
25%,0.157177
50%,0.295544
75%,0.425886
max,1.635822


In [29]:
compute_fraud_score(model, fraud_df, scaler).describe()

,fraud_score
count,7.000000
mean,1.393763
std,0.961767
min,0.442474
25%,0.616691
50%,1.142262
75%,1.987162
max,2.963901


In [30]:
compute_fraud_score(model, fraud_df, scaler)

,fraud_score
0,0.606003
1,0.627380
2,2.306757
3,1.667568
4,0.442474
5,1.142262
6,2.963901


Based on these tables, we will set a threshold of 0.8:

In [31]:
FRAUD_SCORE_THRESHOLD = 0.8

Now we predict the which of the customers in the test set is fraud:

In [32]:
test_df['fraud_score'] = compute_fraud_score(model, test_df[feature_cols], scaler)
test_df['Marker'] = test_df['fraud_score'].apply(lambda score: 'FRAUD' if score >= FRAUD_SCORE_THRESHOLD else 'NOT FRAUD')

Display the `CustomerID` of the people we think are fraud:

In [33]:
test_df.query('Marker == "FRAUD"')['CustomerID'].to_frame()

,CustomerID
9,2010
38,2039
79,2080
81,2082
92,2093
